In [1]:
import pandas as pd
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset = pd.get_dummies(dataset, dtype=int, drop_first=True)
dataset = dataset.drop(['User ID'],axis=1)
dataset.columns

#independent, dependent variable
independent = dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
dependent = dataset[['Purchased']]

#split training and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent, dependent, test_size=0.30, random_state=0)

#preprocessing the training and test set
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train = sc.fit_transform(X_train)
X_test= sc.transform(X_test)

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
param_grid = {
                'C':[10,100,1000,2000,3000],
                'gamma':['auto','scale'],
                'kernel':['linear','rbf','poly','sigmoid']
             }
grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose=3, n_jobs=-1, scoring='f1_weighted')
grid.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


C:\ProgramData\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [15]:
grid_pred = grid.predict(X_test)

In [17]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, grid_pred)

In [21]:
from sklearn.metrics import classification_report
cf_report = classification_report(y_test,grid_pred)
print(cf_matrix)
print(cf_report)

[[73  6]
 [ 4 37]]
              precision    recall  f1-score   support

           0       0.95      0.92      0.94        79
           1       0.86      0.90      0.88        41

    accuracy                           0.92       120
   macro avg       0.90      0.91      0.91       120
weighted avg       0.92      0.92      0.92       120



In [25]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, grid.predict_proba(X_test)[:,1])

0.9654214263661625

In [31]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_pred,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}: 0.9171245421245421


In [27]:
re=grid.cv_results_
table=pd.DataFrame.from_dict(re)

In [29]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.041347,0.013794,0.011253,0.001352,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,21
1,0.035222,0.012327,0.014288,0.004663,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.892857,0.875644,0.841398,0.947015,0.946153,0.900613,0.041029,1
2,0.033469,0.009434,0.016053,0.005549,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.833024,0.799537,0.737557,0.928571,0.888158,0.837369,0.066798,12
3,0.028405,0.014869,0.013602,0.004247,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.769053,0.733523,0.730543,0.753871,0.759910,0.749380,0.014996,32
4,0.024289,0.004547,0.012740,0.002175,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,21
5,0.030144,0.018287,0.014811,0.003788,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.892857,0.875644,0.841398,0.947015,0.946153,0.900613,0.041029,1
6,0.023119,0.006223,0.012228,0.002128,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",0.833024,0.799537,0.737557,0.928571,0.888158,0.837369,0.066798,12
7,0.015001,0.003188,0.009436,0.004202,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.769053,0.733523,0.610390,0.753871,0.759910,0.725349,0.058652,40
8,0.068151,0.007010,0.013433,0.010549,100,auto,linear,"{'C': 100, 'gamma': 'auto', 'kernel': 'linear'}",0.835985,0.782971,0.644599,0.927778,0.890114,0.816289,0.098849,21
9,0.030253,0.010565,0.013394,0.005715,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.855314,0.892857,0.859435,0.929144,0.928571,0.893064,0.031996,3


In [33]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 25
BMI: 21
Children: 2
Sex Male 0 or 1: 1
Smoker Yes 0 or 1: 0


In [35]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]
